# Evaluation Text Simplification

In [1]:
%%capture
!pip install transformers datasets sacrebleu sacremoses

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, load_metric
from collections import Counter

In [3]:
def simplify(text, model, tokenizer):
    encoded_input = tokenizer(text, return_tensors="pt").input_ids
    outputs = model.generate(encoded_input)
    outputs = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return outputs

def get_value(List, index, defualt_value=""):
    return List[index] if len(List) > index else defualt_value

class TextSimplificationModel:
    def __init__(self, tokenizers, models):
        self.tokenizers = tokenizers
        self.models = models

    def simplify_voting_ensemble(self, text, repeat=1):
        output_sentence = []
        simplifications = []
        for model, tokenizer in zip(self.models, self.tokenizers):
            simplification = simplify(text, model, tokenizer)
            simplifications.append(simplification)

        splitted_simplifications = [sentence.split() for sentence in simplifications]
        max_len_sentence = max([len(sentence) for sentence in splitted_simplifications])
        for index in range(max_len_sentence):
            words = [
                        get_value(splitted_simplifications[0], index),
                        get_value(splitted_simplifications[1], index),
                        get_value(splitted_simplifications[2], index),
                    ]
            most_repeated_word, count = Counter(words).most_common(1)[0]
            # ignore empty string and if count less than 2
            if count > repeat and most_repeated_word:
                output_sentence.append(most_repeated_word)
        return " ".join(output_sentence)

In [4]:
checkpoint_1 = "husseinMoh/t5-small-finetuned-text-simplification"
checkpoint_2 = "husseinMoh/bart-base-finetuned-text-simplification"
checkpoint_3 = "husseinMoh/flan-t5-small-finetuned-text-simplification"

tokenizer1 = AutoTokenizer.from_pretrained(checkpoint_1)
tokenizer2 = AutoTokenizer.from_pretrained(checkpoint_2)
tokenizer3 = AutoTokenizer.from_pretrained(checkpoint_3)
tokenizers = [tokenizer1,tokenizer2, tokenizer3]
print("Tokenizers Loaded")

model1 = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_1)
model2 = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_2)
model3 = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_3)
models = [model1, model2, model3]
print("Models Loaded")

Model = TextSimplificationModel(tokenizers, models)

Tokenizers Loaded


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Models Loaded


In [5]:
test_1_input = "Hi NourEldin"
print(f"First input Test example: {test_1_input}")
print(f"Test 1: {Model.simplify_voting_ensemble(test_1_input, repeat=0) = }")
print(f"Test 2: {Model.simplify_voting_ensemble(test_1_input, repeat=1) = }")
print(f"Test 3: {Model.simplify_voting_ensemble(test_1_input, repeat=2) = }")
print()

test_2_input = "The derivation of the equation is demonstrated through various mathematical"
print(f"Second input Test example: {test_2_input}")
print(f"Test 1: {Model.simplify_voting_ensemble(test_2_input, repeat=0) = }")
print(f"Test 2: {Model.simplify_voting_ensemble(test_2_input, repeat=1) = }")
print(f"Test 3: {Model.simplify_voting_ensemble(test_2_input, repeat=2) = }")

First input Test example: Hi NourEldin


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Test 1: Model.simplify_voting_ensemble(test_1_input, repeat=0) = 'Hi NourEldin is the oldest of the tribes of the tribes of'
Test 2: Model.simplify_voting_ensemble(test_1_input, repeat=1) = 'Hi NourEldin'
Test 3: Model.simplify_voting_ensemble(test_1_input, repeat=2) = ''

Second input Test example: The derivation of the equation is demonstrated through various mathematical
Test 1: Model.simplify_voting_ensemble(test_2_input, repeat=0) = 'Different methods of measuring different kinds of molecules are used by'
Test 2: Model.simplify_voting_ensemble(test_2_input, repeat=1) = 'of'
Test 3: Model.simplify_voting_ensemble(test_2_input, repeat=2) = ''


In [6]:
raw_datasets = load_dataset("wiki_auto", "manual", split=["test[:1%]"])[0]
raw_datasets

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/373801 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/73249 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/118074 [00:00<?, ? examples/s]

Dataset wiki_auto downloaded and prepared to /root/.cache/huggingface/datasets/wiki_auto/manual/1.0.0/eeac705719dc9aa2ff180571dfed6c6649588ccdfde8d45a47d2e47e5c5b93af. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['alignment_label', 'normal_sentence_id', 'simple_sentence_id', 'normal_sentence', 'simple_sentence', 'gleu_score'],
    num_rows: 1181
})

In [7]:
metric = load_metric("sari")
def compute_metrics(decoded_preds, decoded_labels, decoded_inputs):
    result = metric.compute(sources=decoded_inputs, predictions=decoded_preds, references=decoded_labels)
    return result

In [8]:
results = []

for row in raw_datasets:
    prediction = [Model.simplify_voting_ensemble(row["normal_sentence"], repeat=0)]
    source = [row["normal_sentence"]]
    reference = [[row["simple_sentence"]]]
    result = compute_metrics(prediction, reference, source)
    results.append(result['sari'])

print(f"SARI {sum(results)/len(results)}")

SARI 55.22108485726729
